# **Deep Learning For Lung Cancer Prediction Based on Transcriptomic Data : a Deformable Convolutional Neural Network**
> Author : **Aymen MERROUCHE**. <br>
> In this notebook, we implement a deformable convolutional network for our binary classification task. First we pre train our MLP on the non Lung cancer dataset. Then, we fine tune it on the lung cancer dataset (we don't keep the final classification layer) :

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import datetime

from utils import *
from train import *
from data_utils import *
from modules.deformable_cnn import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

In [2]:
# device to use, if cuda available then use cuda else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working on : ", device)

Working on :  cuda


In [3]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/def_cnn.yaml', 'r') as stream:
    basic_cnn_args  = yaml.load(stream,Loader=yaml.Loader)

## **1 - Pre-Training on the Non Lung Dataset :**

### **1 - 1 - Get the Data :**

In [4]:
%%time
# Getting the data
# dataset
non_lung_dataset = TranscriptomicImagesDatasetNonLung(data_paths_args["path_to_pan_cancer_hdf5_files"], data_paths_args["path_to_treemap_images"])
non_lung_dataloader_train, non_lung_dataloader_validation = get_data_loaders(non_lung_dataset, batch_size_train = basic_cnn_args["batch_size_pt"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_pt"])

CPU times: user 4.75 s, sys: 1.05 s, total: 5.8 s
Wall time: 3.04 s


### **1 - 2 - Network, Criterion and Training :**

In [5]:
# network
net = DeformConvNet().to(device).double()

# loss and optimizer  
criterion = FocalLoss(gamma=3).to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**basic_cnn_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('/tempory/transcriptomic_data/pre_trained_def_cnn_checkpt.pt')
# Tensorboard summary writer
if basic_cnn_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
    
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, non_lung_dataloader_train, non_lung_dataloader_validation, basic_cnn_args['epochs'], writer=writer)
if not basic_cnn_args['no_tensorboard']:
    writer.close()

Epoch 1/50:   0%|          | 1/646 [00:00<01:16,  8.40it/s, loss=7.8625e-02]

Experimenting with : 
 	epochs=50_batch_size_pt=8_lr_pt=0.001_batch_size_ft=8_lr_ft=0.001_no_tensorboard=True
Training on GPU 



Epoch 2/50:   0%|          | 1/646 [00:00<01:04,  9.94it/s, loss=9.2470e-02]

Epoch 1/50, Train Loss: 8.4050e-02, Test Loss: 0.0829
Epoch 1/50, Train Accuracy: 55.77%, Test Accuracy: 60.80%
Epoch 1/50, Train AUC: 57.66%, Test AUC: 54.69%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.89      0.63      0.74      2248
      Cancer       0.13      0.43      0.20       295

    accuracy                           0.61      2543
   macro avg       0.51      0.53      0.47      2543
weighted avg       0.81      0.61      0.68      2543



Epoch 3/50:   0%|          | 1/646 [00:00<01:12,  8.92it/s, loss=7.1253e-02]

...

Epoch 20/50, Train Loss: 1.0751e-02, Test Loss: 0.0766
Epoch 20/50, Train Accuracy: 88.29%, Test Accuracy: 69.76%
Epoch 20/50, Train AUC: 96.29%, Test AUC: 60.87%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.90      0.74      0.81      2248
      Cancer       0.16      0.39      0.23       295

    accuracy                           0.70      2543
   macro avg       0.53      0.56      0.52      2543
weighted avg       0.82      0.70      0.74      2543



Epoch 22/50:   0%|          | 2/646 [00:00<01:01, 10.41it/s, loss=7.7205e-03]

Epoch 21/50, Train Loss: 2.3619e-02, Test Loss: 0.0822
Epoch 21/50, Train Accuracy: 82.72%, Test Accuracy: 66.81%
Epoch 21/50, Train AUC: 90.11%, Test AUC: 61.14%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.90      0.70      0.79      2248
      Cancer       0.15      0.39      0.21       295

    accuracy                           0.67      2543
   macro avg       0.52      0.55      0.50      2543
weighted avg       0.81      0.67      0.72      2543



Epoch 22/50:  49%|████▊     | 314/646 [00:29<00:30, 10.80it/s, loss=3.3398e-02]


KeyboardInterrupt: 

## **2 - Fine-Tuning on the Lung dataset :**

### **2 - 1 - Load Pre-Trained Model :**

In [6]:
# Load the pretrained Model
net =  DeformConvNet().to(device).double()
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
# path where the pre-trained model is saved : defined above+"_best"
savepath = Path('/tempory/transcriptomic_data/pre_trained_def_cnn_checkpt_best.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
checkpoint.load()
pretrained = checkpoint.model

### **2 - 2 - Get the Data :**

In [7]:
%%time
# Getting the data
# dataset
lung_dataset = TranscriptomicImagesDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"], data_paths_args["path_to_treemap_images"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = basic_cnn_args["batch_size_ft"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_ft"])

CPU times: user 580 ms, sys: 96.5 ms, total: 677 ms
Wall time: 330 ms


### **2 - 3 - Fine Tuning Procedure :**

In [8]:
# Beginnig Of Transfer Learnig Procedure
net = get_deform_cnn_fine_tune(pretrained, freeze_cnn=False, freeze_filter=[nn.Conv2d])
net = net.to(device).double()
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
savepath = Path('/tempory/transcriptomic_data/fine_tuned_def_cnn_checkpt.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, basic_cnn_args['epochs'])

Epoch 1/50:   1%|▏         | 1/72 [00:00<00:07,  8.92it/s, loss=1.4041e-01]

Training on GPU 



Epoch 2/50:   3%|▎         | 2/72 [00:00<00:06, 10.58it/s, loss=1.2665e-01]

Epoch 1/50, Train Loss: 1.4972e-01, Test Loss: 0.1834
Epoch 1/50, Train Accuracy: 65.52%, Test Accuracy: 52.08%
Epoch 1/50, Train AUC: 66.48%, Test AUC: 46.51%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.91      0.53      0.67       257
      Cancer       0.08      0.44      0.14        25

    accuracy                           0.52       282
   macro avg       0.49      0.48      0.40       282
weighted avg       0.83      0.52      0.62       282



Epoch 3/50:   3%|▎         | 2/72 [00:00<00:06, 10.54it/s, loss=1.8998e-01]

...

Epoch 4/50, Train Loss: 8.4315e-02, Test Loss: 0.1396
Epoch 4/50, Train Accuracy: 91.49%, Test Accuracy: 72.57%
Epoch 4/50, Train AUC: 95.01%, Test AUC: 52.06%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.91      0.77      0.83       257
      Cancer       0.08      0.20      0.11        25

    accuracy                           0.72       282
   macro avg       0.49      0.49      0.47       282
weighted avg       0.83      0.72      0.77       282



Epoch 6/50:   3%|▎         | 2/72 [00:00<00:06, 10.58it/s, loss=5.0780e-02]

Epoch 5/50, Train Loss: 5.8970e-02, Test Loss: 0.1101
Epoch 5/50, Train Accuracy: 91.22%, Test Accuracy: 77.78%
Epoch 5/50, Train AUC: 94.50%, Test AUC: 50.13%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.91      0.84      0.87       257
      Cancer       0.07      0.12      0.09        25

    accuracy                           0.77       282
   macro avg       0.49      0.48      0.48       282
weighted avg       0.83      0.77      0.80       282



Epoch 6/50:  47%|████▋     | 34/72 [00:03<00:03, 10.84it/s, loss=1.3203e-01]


KeyboardInterrupt: 